# Group Work 2

### Derivative Pricing

Black-Scholes closed-form solution

In [ ]:
import numpy as np
import scipy.stats as si

def black_scholes(S, K, T, r, sigma, option_type="call"):
    """
    Black-Scholes formula for European options.

    Parameters:
    S (float): Initial stock price
    K (float): Strike price
    T (float): Time to maturity (in years)
    r (float): Risk-free interest rate (decimal)
    sigma (float): Volatility of the underlying asset
    option_type (str): "call" for call option, "put" for put option

    Returns:
    float: Option price
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == "call":
        price = S * si.norm.cdf(d1) - K * np.exp(-r * T) * si.norm.cdf(d2)
    elif option_type == "put":
        price = K * np.exp(-r * T) * si.norm.cdf(-d2) - S * si.norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type. Choose 'call' or 'put'.")

    return price

# Example Usage
S0 = 100  # Initial stock price
K = 100   # Strike price (ATM option)
r = 0.05  # Risk-free rate (5%)
sigma = 0.20  # Volatility (20%)
T = 3/12  # 3 months converted to years

call_price = black_scholes(S0, K, T, r, sigma, "call")
put_price = black_scholes(S0, K, T, r, sigma, "put")

print(f"European Call Price: {call_price:.4f}")
print(f"European Put Price: {put_price:.4f}")


European Call Price: 4.6150
European Put Price: 3.3728


In [ ]:
import numpy as np
import scipy.stats as si

def delta(S, K, T, r, sigma, option_type="call"):
    """
    Computes the Delta of a European option using Black-Scholes formula.

    Parameters:
    S (float): Initial stock price
    K (float): Strike price
    T (float): Time to maturity (in years)
    r (float): Risk-free interest rate (decimal)
    sigma (float): Volatility of the underlying asset
    option_type (str): "call" for call option, "put" for put option

    Returns:
    float: Delta value
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))

    if option_type == "call":
        return si.norm.cdf(d1)  # N(d1) for Call
    elif option_type == "put":
        return si.norm.cdf(d1) - 1  # N(d1) - 1 for Put
    else:
        raise ValueError("Invalid option type. Choose 'call' or 'put'.")

# Given parameters
S0 = 100  # Initial stock price
K = 100   # Strike price (ATM option)
r = 0.05  # Risk-free rate (5%)
sigma = 0.20  # Volatility (20%)
T = 3/12  # 3 months converted to years

# Compute Delta
call_delta = delta(S0, K, T, r, sigma, "call")
put_delta = delta(S0, K, T, r, sigma, "put")

print(f"Call Delta: {call_delta:.4f}")
print(f"Put Delta: {put_delta:.4f}")


Call Delta: 0.5695
Put Delta: -0.4305


In [ ]:
def vega(S, K, T, r, sigma):
    """
    Computes the Vega of a European option using Black-Scholes formula.

    Parameters:
    S (float): Initial stock price
    K (float): Strike price
    T (float): Time to maturity (in years)
    r (float): Risk-free interest rate (decimal)
    sigma (float): Volatility of the underlying asset

    Returns:
    float: Vega value (price change per 1% change in volatility)
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return S * np.sqrt(T) * si.norm.pdf(d1)  # PDF of standard normal distribution

# Compute Vega using analytical formula
vega_analytical = vega(S0, K, T, r, sigma)

print(f"Vega (Analytical): {vega_analytical:.4f}")


Vega (Analytical): 19.6440


In [ ]:
# Compute Vega numerically by increasing volatility
sigma_new = 0.25  # Increase volatility from 20% to 25%
call_price_new = black_scholes(S0, K, T, r, sigma_new, "call")
put_price_new = black_scholes(S0, K, T, r, sigma_new, "put")

# Numerical Vega computation
vega_numerical = (call_price_new - call_price) / (sigma_new - sigma)

print(f"Vega (Numerical): {vega_numerical:.4f}")


Vega (Numerical): 19.6681


Monte-Carlo methods with regular GBM Process

In [ ]:
import numpy as np

def monte_carlo_american_call(S0, K, T, r, sigma, num_simulations=100000, num_steps=90):
    """
    Monte Carlo simulation to price an American Call option using a GBM process.

    Parameters:
    S0 (float): Initial stock price
    K (float): Strike price
    T (float): Time to maturity (in years)
    r (float): Risk-free interest rate (decimal)
    sigma (float): Volatility of the underlying asset
    num_simulations (int): Number of Monte Carlo simulations
    num_steps (int): Number of time steps (daily)

    Returns:
    float: Estimated American Call price
    """
    dt = T / num_steps  # Time step size
    discount_factor = np.exp(-r * T)  # Discounting factor

    # Simulate stock price paths using Geometric Brownian Motion (GBM)
    S = np.zeros((num_simulations, num_steps + 1))
    S[:, 0] = S0  # Initial stock price

    for t in range(1, num_steps + 1):
        Z = np.random.standard_normal(num_simulations)  # Standard normal random values
        S[:, t] = S[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)

    # Compute option payoff at maturity
    payoffs = np.maximum(S[:, -1] - K, 0)  # Call option payoff

    # Monte Carlo estimated price (discounted expectation)
    call_price_mc = discount_factor * np.mean(payoffs)

    return call_price_mc

def monte_carlo_delta_vega(S0, K, T, r, sigma, num_simulations=100000, num_steps=90, dS=1, dSigma=0.05):
    """
    Computes Delta and Vega for an American Call option using Monte Carlo simulations.

    Parameters:
    S0 (float): Initial stock price
    K (float): Strike price
    T (float): Time to maturity (in years)
    r (float): Risk-free interest rate (decimal)
    sigma (float): Volatility of the underlying asset
    num_simulations (int): Number of Monte Carlo simulations
    num_steps (int): Number of time steps (daily)
    dS (float): Small change in stock price for Delta calculation
    dSigma (float): Small change in volatility for Vega calculation

    Returns:
    tuple: (Delta, Vega) for the American Call option
    """
    # Compute base option price
    call_price_base = monte_carlo_american_call(S0, K, T, r, sigma, num_simulations, num_steps)

    # Compute option prices for perturbed stock prices
    call_price_up = monte_carlo_american_call(S0 + dS, K, T, r, sigma, num_simulations, num_steps)
    call_price_down = monte_carlo_american_call(S0 - dS, K, T, r, sigma, num_simulations, num_steps)

    # Compute Delta using bump-and-revalue method
    delta_mc = (call_price_up - call_price_down) / (2 * dS)

    # Compute option price for increased volatility
    call_price_high_vol = monte_carlo_american_call(S0, K, T, r, sigma + dSigma, num_simulations, num_steps)

    # Compute Vega using bump-and-revalue method
    vega_mc = (call_price_high_vol - call_price_base) / dSigma

    return delta_mc, vega_mc

# Given parameters
S0 = 100  # Initial stock price
K = 100   # Strike price (ATM option)
r = 0.05  # Risk-free rate (5%)
sigma = 0.20  # Volatility (20%)
T = 3/12  # 3 months converted to years

# Compute American Call price using Monte Carlo
american_call_price = monte_carlo_american_call(S0, K, T, r, sigma)

# Compute Delta and Vega using Monte Carlo
delta_mc, vega_mc = monte_carlo_delta_vega(S0, K, T, r, sigma)

# Output results
print(f"Estimated American Call Price (Monte Carlo): {american_call_price:.4f}")
print(f"Monte Carlo Delta: {delta_mc:.4f}")
print(f"Monte Carlo Vega: {vega_mc:.4f}")


Estimated American Call Price (Monte Carlo): 4.6195
Monte Carlo Delta: 0.5672
Monte Carlo Vega: 19.9285


In [ ]:
import numpy as np
import scipy.stats as si

# Black-Scholes function for European options
def black_scholes(S, K, T, r, sigma, option_type="call"):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == "call":
        price = S * si.norm.cdf(d1) - K * np.exp(-r * T) * si.norm.cdf(d2)
    elif option_type == "put":
        price = K * np.exp(-r * T) * si.norm.cdf(-d2) - S * si.norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type. Choose 'call' or 'put'.")

    return price

# Delta function for European options
def delta(S, K, T, r, sigma, option_type="call"):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))

    if option_type == "call":
        return si.norm.cdf(d1)  # N(d1) for Call
    elif option_type == "put":
        return si.norm.cdf(d1) - 1  # N(d1) - 1 for Put

# Portfolio Delta Calculation
def portfolio_delta(call_delta, put_delta, case="buy_both"):
    """
    Computes the portfolio delta for different strategies.

    Parameters:
    call_delta (float): Delta of the European call option
    put_delta (float): Delta of the European put option
    case (str): "buy_both" for buying both call and put,
                "buy_call_sell_put" for buying the call and selling the put

    Returns:
    float: Portfolio Delta
    """
    if case == "buy_both":
        return call_delta + put_delta
    elif case == "buy_call_sell_put":
        return call_delta - put_delta
    else:
        raise ValueError("Invalid case. Choose 'buy_both' or 'buy_call_sell_put'.")

# Hedging Strategy
def hedging_strategy(portfolio_delta):
    """
    Determines the hedging strategy based on portfolio delta.

    Parameters:
    portfolio_delta (float): The computed portfolio delta

    Returns:
    str: Suggested hedging strategy
    """
    if abs(portfolio_delta) < 0.05:
        return "Portfolio is nearly delta-neutral. No significant hedging required."
    elif portfolio_delta > 0:
        return f"Short {portfolio_delta:.4f} shares of the underlying stock per option contract to hedge."
    else:
        return f"Long {-portfolio_delta:.4f} shares of the underlying stock per option contract to hedge."

# Given parameters
S0 = 100  # Initial stock price
r = 0.05  # Risk-free rate (5%)
sigma = 0.20  # Volatility (20%)
T = 3/12  # 3 months converted to years

# Strike prices for different moneyness levels
K_call = 110  # 110% moneyness for call
K_put = 95    # 95% moneyness for put

# Compute Prices
call_price = black_scholes(S0, K_call, T, r, sigma, "call")
put_price = black_scholes(S0, K_put, T, r, sigma, "put")

# Compute Delta values
call_delta = delta(S0, K_call, T, r, sigma, "call")
put_delta = delta(S0, K_put, T, r, sigma, "put")

# Compute Portfolio Deltas
portfolio_delta_1 = portfolio_delta(call_delta, put_delta, "buy_both")
portfolio_delta_2 = portfolio_delta(call_delta, put_delta, "buy_call_sell_put")

# Compute Hedging Strategies
hedging_1 = hedging_strategy(portfolio_delta_1)
hedging_2 = hedging_strategy(portfolio_delta_2)

# Output results
print(f"Call Price (110% Moneyness): {call_price:.4f}")
print(f"Put Price (95% Moneyness): {put_price:.4f}")
print(f"Call Delta: {call_delta:.4f}")
print(f"Put Delta: {put_delta:.4f}\n")

print(f"Portfolio 1 (Buy Call & Put) Delta: {portfolio_delta_1:.4f}")
print(f"Hedging Strategy for Portfolio 1: {hedging_1}\n")

print(f"Portfolio 2 (Buy Call, Sell Put) Delta: {portfolio_delta_2:.4f}")
print(f"Hedging Strategy for Portfolio 2: {hedging_2}")


Call Price (110% Moneyness): 1.1911
Put Price (95% Moneyness): 1.5343
Call Delta: 0.2183
Put Delta: -0.2457

Portfolio 1 (Buy Call & Put) Delta: -0.0275
Hedging Strategy for Portfolio 1: Portfolio is nearly delta-neutral. No significant hedging required.

Portfolio 2 (Buy Call, Sell Put) Delta: 0.4640
Hedging Strategy for Portfolio 2: Short 0.4640 shares of the underlying stock per option contract to hedge.


In [1]:
import numpy as np
import scipy.stats as si

def black_scholes(S, K, T, r, sigma, option_type="call"):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == "call":
        price = S * si.norm.cdf(d1) - K * np.exp(-r * T) * si.norm.cdf(d2)
    elif option_type == "put":
        price = K * np.exp(-r * T) * si.norm.cdf(-d2) - S * si.norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'")

    return price

def monte_carlo_option(S, K, T, r, sigma, simulations=10000, option_type="call"):
    np.random.seed(42)
    Z = np.random.standard_normal(simulations)
    ST = S * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * Z)

    if option_type == "call":
        payoffs = np.maximum(ST - K, 0)
    elif option_type == "put":
        payoffs = np.maximum(K - ST, 0)
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'")

    price = np.exp(-r * T) * np.mean(payoffs)
    return price

def check_put_call_parity(call_price, put_price, S, K, r, T):
    parity_lhs = call_price - put_price
    parity_rhs = S - K * np.exp(-r * T)
    return np.isclose(parity_lhs, parity_rhs), parity_lhs, parity_rhs

# Example usage
S = 100  # Stock price
K = 100  # Strike price
T = 1    # Time to maturity (1 year)
r = 0.05 # Risk-free rate
sigma = 0.2 # Volatility

# Compute Black-Scholes Prices
bs_call = black_scholes(S, K, T, r, sigma, "call")
bs_put = black_scholes(S, K, T, r, sigma, "put")

# Compute Monte Carlo Prices
mc_call = monte_carlo_option(S, K, T, r, sigma, option_type="call")
mc_put = monte_carlo_option(S, K, T, r, sigma, option_type="put")

# Check Put-Call Parity
parity_check, lhs, rhs = check_put_call_parity(bs_call, bs_put, S, K, r, T)

print(f"Black-Scholes Call: {bs_call:.4f}, Put: {bs_put:.4f}")
print(f"Monte Carlo Call: {mc_call:.4f}, Put: {mc_put:.4f}")
print(f"Put-Call Parity Check: {parity_check}, LHS: {lhs:.4f}, RHS: {rhs:.4f}")


Black-Scholes Call: 10.4506, Put: 5.5735
Monte Carlo Call: 10.4502, Put: 5.6017
Put-Call Parity Check: True, LHS: 4.8771, RHS: 4.8771
